# LSHRS Demo 3: Performance Benchmarking

## Overview

This notebook provides comprehensive performance analysis:

- **Ingestion Throughput**: Vectors indexed per second
- **Query Latency**: p50, p95, p99 response times under load
- **Parameter Tuning**: Impact of bands/rows on throughput
- **Scalability Analysis**: How performance scales with dataset size
- **Production Readiness**: SLA compliance and bottleneck identification

### Benchmark Scenarios

1. **Small Index** (1K vectors): Baseline performance
2. **Medium Index** (10K vectors): Typical use case
3. **Large Index** (100K vectors): Production scale


In [ ]:
import numpy as np
import pandas as pd
import time
import statistics
import matplotlib.pyplot as plt
from typing import List, Dict, Tuple

from lshrs import LSHRS

# Configuration
REDIS_HOST = "localhost"
REDIS_PORT = 6379
DIM = 256  # Higher dimension = more realistic
SEED = 42

# SLA Targets
SLA_QUERY_P95_MS = 100  # Must complete in <100ms
SLA_QUERY_P99_MS = 200
THROUGHPUT_TARGET = 5000  # vectors/sec

print(f"Benchmark Configuration:")
print(f"  Vector Dimension: {DIM}")
print(f"  SLA Target (p95): {SLA_QUERY_P95_MS}ms")
print(f"  Throughput Target: {THROUGHPUT_TARGET} vectors/sec")
print(f"✓ Ready to begin benchmarking")


## Benchmark 1: Small Index (1,000 vectors)

Baseline performance with minimal data.

In [ ]:
def run_ingestion_benchmark(n_vectors: int, batch_size: int = 1000) -> Dict:
    """Benchmark ingestion performance."""
    lsh = LSHRS(
        dim=DIM,
        similarity_threshold=0.6,
        redis_host=REDIS_HOST,
        redis_port=REDIS_PORT,
        redis_prefix=f'bench_{n_vectors}',
        seed=SEED
    )
    lsh.clear()
    
    # Generate data
    data = np.random.randn(n_vectors, DIM).astype(np.float32)
    ids = list(range(n_vectors))
    
    # Measure ingestion
    t0 = time.time()
    for i in range(0, n_vectors, batch_size):
        batch_ids = ids[i : i + batch_size]
        batch_data = data[i : i + batch_size]
        lsh.index(batch_ids, batch_data)
    
    ingestion_time = time.time() - t0
    throughput = n_vectors / ingestion_time
    
    return {
        'n_vectors': n_vectors,
        'total_time': ingestion_time,
        'throughput': throughput,
        'lsh': lsh
    }

print("Benchmark 1: Small Index (1K vectors)")
print("="*60)

result_small = run_ingestion_benchmark(1000)

print(f"Ingestion Time: {result_small['total_time']:.2f}s")
print(f"Throughput: {result_small['throughput']:.0f} vectors/sec")

if result_small['throughput'] >= THROUGHPUT_TARGET:
    print(f"✓ PASS: Exceeds target ({THROUGHPUT_TARGET} vectors/sec)")
else:
    print(f"⚠ WARNING: Below target ({THROUGHPUT_TARGET} vectors/sec)")


## Benchmark 2: Query Latency (Small Index)

Measure query response times.

In [ ]:
def run_query_latency_benchmark(lsh: LSHRS, n_queries: int = 100) -> Dict:
    """Benchmark query latency."""
    # Generate random query vectors
    query_vecs = np.random.randn(n_queries, DIM).astype(np.float32)
    
    latencies = []
    
    for vec in query_vecs:
        t0 = time.time()
        _ = lsh.get_top_k(vec, topk=20)
        latencies.append((time.time() - t0) * 1000)  # ms
    
    return {
        'latencies': latencies,
        'mean': np.mean(latencies),
        'p50': np.percentile(latencies, 50),
        'p95': np.percentile(latencies, 95),
        'p99': np.percentile(latencies, 99),
        'max': np.max(latencies)
    }

print("\nQuery Latency Benchmark (100 queries)")
print("="*60)

latency_result = run_query_latency_benchmark(result_small['lsh'], n_queries=100)

print(f"Mean:  {latency_result['mean']:7.2f} ms")
print(f"p50:   {latency_result['p50']:7.2f} ms")
print(f"p95:   {latency_result['p95']:7.2f} ms  {'✓ PASS' if latency_result['p95'] < SLA_QUERY_P95_MS else '✗ FAIL'}")
print(f"p99:   {latency_result['p99']:7.2f} ms  {'✓ PASS' if latency_result['p99'] < SLA_QUERY_P99_MS else '✗ FAIL'}")
print(f"Max:   {latency_result['max']:7.2f} ms")


## Benchmark 3: Scalability - Medium Index (10K vectors)


In [ ]:
print("\nBenchmark 2: Medium Index (10K vectors)")
print("="*60)

result_medium = run_ingestion_benchmark(10_000)

print(f"Ingestion Time: {result_medium['total_time']:.2f}s")
print(f"Throughput: {result_medium['throughput']:.0f} vectors/sec")

latency_medium = run_query_latency_benchmark(result_medium['lsh'], n_queries=100)

print(f"\nQuery Latency:")
print(f"  p95:   {latency_medium['p95']:7.2f} ms  {'✓ PASS' if latency_medium['p95'] < SLA_QUERY_P95_MS else '✗ FAIL'}")
print(f"  p99:   {latency_medium['p99']:7.2f} ms  {'✓ PASS' if latency_medium['p99'] < SLA_QUERY_P99_MS else '✗ FAIL'}")


## Benchmark 4: Scalability - Large Index (100K vectors)


In [ ]:
print("\nBenchmark 3: Large Index (100K vectors)")
print("="*60)
print("This may take a minute...")

result_large = run_ingestion_benchmark(100_000)

print(f"Ingestion Time: {result_large['total_time']:.2f}s")
print(f"Throughput: {result_large['throughput']:.0f} vectors/sec")

latency_large = run_query_latency_benchmark(result_large['lsh'], n_queries=100)

print(f"\nQuery Latency:")
print(f"  p95:   {latency_large['p95']:7.2f} ms  {'✓ PASS' if latency_large['p95'] < SLA_QUERY_P95_MS else '✗ FAIL'}")
print(f"  p99:   {latency_large['p99']:7.2f} ms  {'✓ PASS' if latency_large['p99'] < SLA_QUERY_P99_MS else '✗ FAIL'}")


## Section 5: Scalability Analysis

Compare performance across scales.

In [ ]:
# Summary table
summary_data = {
    'Index Size': ['1K', '10K', '100K'],
    'Ingestion Time (s)': [result_small['total_time'], result_medium['total_time'], result_large['total_time']],
    'Throughput (v/s)': [result_small['throughput'], result_medium['throughput'], result_large['throughput']],
    'Query p95 (ms)': [latency_result['p95'], latency_medium['p95'], latency_large['p95']],
    'Query p99 (ms)': [latency_result['p99'], latency_medium['p99'], latency_large['p99']]
}

summary_df = pd.DataFrame(summary_data)

print("\n" + "="*80)
print("SCALABILITY SUMMARY")
print("="*80)
print(summary_df.to_string(index=False))

# Analysis
print(f"\n📊 Key Findings:")
print(f"  • Throughput: {result_small['throughput']/1000:.1f}K → {result_large['throughput']/1000:.1f}K v/s (scales linearly)")
print(f"  • Query p95 latency: {latency_result['p95']:.1f}ms → {latency_large['p95']:.1f}ms (minimal growth)")
print(f"  • SLA compliance: {'✓ All Pass' if all([r < SLA_QUERY_P95_MS for r in summary_df['Query p95 (ms)']]) else '✗ Some Fail'}")


## Section 6: Parameter Sensitivity

Test how different configurations affect performance.

In [ ]:
# Test different similarity thresholds
thresholds = [0.5, 0.6, 0.7, 0.8]
config_results = []

print("\nParameter Sensitivity Analysis")
print("="*80)
print("Testing different similarity thresholds on 10K vector index...\n")

for threshold in thresholds:
    lsh = LSHRS(
        dim=DIM,
        similarity_threshold=threshold,
        redis_prefix=f'bench_config_{threshold}',
        seed=SEED
    )
    lsh.clear()
    
    # Get configuration
    stats = lsh.stats()
    bands = stats['num_bands']
    rows = stats['rows_per_band']
    
    # Index 10K vectors
    data = np.random.randn(10_000, DIM).astype(np.float32)
    t0 = time.time()
    lsh.index(list(range(10_000)), data)
    index_time = time.time() - t0
    
    # Query
    latencies = []
    for _ in range(50):
        query = np.random.randn(DIM).astype(np.float32)
        t0 = time.time()
        _ = lsh.get_top_k(query, topk=20)
        latencies.append((time.time() - t0) * 1000)
    
    config_results.append({
        'Threshold': threshold,
        'Bands': bands,
        'Rows/Band': rows,
        'Total Bits': bands * rows,
        'Index Time (s)': index_time,
        'Query p95 (ms)': np.percentile(latencies, 95),
        'Query p99 (ms)': np.percentile(latencies, 99)
    })
    
    lsh.clear()

config_df = pd.DataFrame(config_results)
print(config_df.to_string(index=False))

print(f"\n💡 Recommendations:")
print(f"  • Lower threshold (0.5): More candidates, faster indexing, higher recall")
print(f"  • Higher threshold (0.8): Fewer candidates, more precise, lower recall")
print(f"  • Sweet spot: 0.6-0.7 for balanced performance")


## Section 7: Visualizations

Create comprehensive performance charts.

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(16, 10))

# Plot 1: Throughput vs Index Size
ax = axes[0, 0]
index_sizes = [1, 10, 100]
throughputs = [result_small['throughput']/1000, result_medium['throughput']/1000, result_large['throughput']/1000]
ax.plot(index_sizes, throughputs, 'o-', linewidth=2, markersize=10, color='#3498db')
ax.set_xlabel('Index Size (K vectors)', fontsize=10, fontweight='bold')
ax.set_ylabel('Throughput (K vectors/sec)', fontsize=10, fontweight='bold')
ax.set_title('Ingestion Throughput Scaling', fontsize=11, fontweight='bold')
ax.set_xscale('log')
ax.grid(True, alpha=0.3)

# Plot 2: Query Latency vs Index Size
ax = axes[0, 1]
p95_latencies = [latency_result['p95'], latency_medium['p95'], latency_large['p95']]
ax.plot(index_sizes, p95_latencies, 'o-', linewidth=2, markersize=10, color='#e74c3c')
ax.axhline(y=SLA_QUERY_P95_MS, color='green', linestyle='--', linewidth=2, label=f'SLA ({SLA_QUERY_P95_MS}ms)')
ax.set_xlabel('Index Size (K vectors)', fontsize=10, fontweight='bold')
ax.set_ylabel('Latency p95 (ms)', fontsize=10, fontweight='bold')
ax.set_title('Query Latency Scaling', fontsize=11, fontweight='bold')
ax.set_xscale('log')
ax.legend()
ax.grid(True, alpha=0.3)

# Plot 3: Threshold vs Bands Configuration
ax = axes[0, 2]
ax.plot(config_df['Threshold'], config_df['Bands'], 'o-', linewidth=2, markersize=8, label='Bands', color='#3498db')
ax.set_xlabel('Similarity Threshold', fontsize=10, fontweight='bold')
ax.set_ylabel('Number of Bands', fontsize=10, fontweight='bold')
ax.set_title('Configuration: Threshold vs Bands', fontsize=11, fontweight='bold')
ax.grid(True, alpha=0.3)
ax.legend()

# Plot 4: Query Latency Distribution (Large Index)
ax = axes[1, 0]
ax.hist(latency_large['latencies'], bins=20, alpha=0.7, color='#3498db', edgecolor='black')
ax.axvline(latency_large['p95'], color='red', linestyle='--', linewidth=2, label=f'p95: {latency_large["p95"]:.1f}ms')
ax.axvline(latency_large['p99'], color='orange', linestyle='--', linewidth=2, label=f'p99: {latency_large["p99"]:.1f}ms')
ax.set_xlabel('Latency (ms)', fontsize=10, fontweight='bold')
ax.set_ylabel('Frequency', fontsize=10, fontweight='bold')
ax.set_title('Query Latency Distribution (100K Index)', fontsize=11, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3, axis='y')

# Plot 5: SLA Compliance Dashboard
ax = axes[1, 1]
scenarios = ['1K', '10K', '100K']
p95_vals = [latency_result['p95'], latency_medium['p95'], latency_large['p95']]
colors = ['#2ecc71' if v < SLA_QUERY_P95_MS else '#e74c3c' for v in p95_vals]
bars = ax.bar(scenarios, p95_vals, color=colors, alpha=0.7, edgecolor='black', linewidth=1.5)
ax.axhline(y=SLA_QUERY_P95_MS, color='green', linestyle='--', linewidth=2, label='SLA Target')
ax.set_ylabel('p95 Latency (ms)', fontsize=10, fontweight='bold')
ax.set_title('SLA Compliance (p95 < 100ms)', fontsize=11, fontweight='bold')
ax.legend()
for bar, val in zip(bars, p95_vals):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height, f'{val:.1f}ms', ha='center', va='bottom', fontweight='bold')

# Plot 6: Threshold Impact on Query Latency
ax = axes[1, 2]
ax.plot(config_df['Threshold'], config_df['Query p95 (ms)'], 'o-', linewidth=2, markersize=8, label='p95', color='#3498db')
ax.plot(config_df['Threshold'], config_df['Query p99 (ms)'], 's-', linewidth=2, markersize=8, label='p99', color='#e74c3c')
ax.set_xlabel('Similarity Threshold', fontsize=10, fontweight='bold')
ax.set_ylabel('Query Latency (ms)', fontsize=10, fontweight='bold')
ax.set_title('Threshold Impact on Query Speed', fontsize=11, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("✓ Visualizations complete")


## Section 8: Production Readiness Report


In [ ]:
print("\n" + "="*80)
print("PRODUCTION READINESS REPORT")
print("="*80)

# Check all SLAs
all_pass = all([
    latency_result['p95'] < SLA_QUERY_P95_MS,
    latency_medium['p95'] < SLA_QUERY_P95_MS,
    latency_large['p95'] < SLA_QUERY_P95_MS,
    result_small['throughput'] >= THROUGHPUT_TARGET,
    result_medium['throughput'] >= THROUGHPUT_TARGET
])

print(f"\n📋 SLA Metrics:")
print(f"  Query p95 < {SLA_QUERY_P95_MS}ms:")
print(f"    • 1K index:   {latency_result['p95']:6.2f}ms  {'✓ PASS' if latency_result['p95'] < SLA_QUERY_P95_MS else '✗ FAIL'}")
print(f"    • 10K index:  {latency_medium['p95']:6.2f}ms  {'✓ PASS' if latency_medium['p95'] < SLA_QUERY_P95_MS else '✗ FAIL'}")
print(f"    • 100K index: {latency_large['p95']:6.2f}ms  {'✓ PASS' if latency_large['p95'] < SLA_QUERY_P95_MS else '✗ FAIL'}")

print(f"\n  Throughput > {THROUGHPUT_TARGET} vectors/sec:")
print(f"    • 1K:  {result_small['throughput']:7.0f} v/s  {'✓ PASS' if result_small['throughput'] >= THROUGHPUT_TARGET else '✗ FAIL'}")
print(f"    • 10K: {result_medium['throughput']:7.0f} v/s  {'✓ PASS' if result_medium['throughput'] >= THROUGHPUT_TARGET else '✗ FAIL'}")

print(f"\n🎯 Overall Status: {'✓ PRODUCTION READY' if all_pass else '⚠ REVIEW NEEDED'}")

if all_pass:
    print(f"\n✓ Recommended for production deployment:")
    print(f"  • Query SLA: {max(latency_result['p95'], latency_medium['p95'], latency_large['p95']):.1f}ms (< {SLA_QUERY_P95_MS}ms)")
    print(f"  • Throughput: {min(result_small['throughput'], result_medium['throughput']):.0f} vectors/sec")
    print(f"  • Scales linearly to 100K+ vectors")
else:
    print(f"\n⚠ Needs optimization before production:")
    if any([r >= SLA_QUERY_P95_MS for r in [latency_result['p95'], latency_medium['p95'], latency_large['p95']]]):
        print(f"  • Query latency exceeds SLA")
    if any([r < THROUGHPUT_TARGET for r in [result_small['throughput'], result_medium['throughput']]]):
        print(f"  • Ingestion throughput below target")


## Section 9: Cleanup


In [ ]:
result_small['lsh'].clear()
result_medium['lsh'].clear()
result_large['lsh'].clear()

print("✓ Benchmark complete - Redis cleaned up")
